In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

from tqdm import tqdm

BATCH_SIZE = 16

# source: https://www.tensorflow.org/tutorials/structured_data/time_series
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                 train_df=None, val_df=None, test_df=None,
                 label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                          enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[
            self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[
            self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]]
                    for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='sales', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(
                    plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()
        plt.xlabel('Time [h]')

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result


def fourier_transform(dataframe:pd.DataFrame) -> pd.DataFrame:
    temp = dataframe.copy()
    date_time = pd.to_datetime(temp.pop("date"), format='%Y-%m-%d')

    day = 24*60*60
    year = (365.2425)*day
    
    timestamp_s = date_time.map(pd.Timestamp.timestamp)
    
    temp['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    temp['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    temp = temp.set_index(date_time)
    return temp

def make_data(dataframe:pd.DataFrame):
    data = fourier_transform(dataframe)
    dummy = data.copy().drop(labels=['store', 'item'], axis=1)
    window = WindowGenerator(input_width=30, label_width=1, shift=1, train_df=dummy, label_columns=['sales'])
    stores = data["store"].unique()

    # data
    x1 = [[]] # store, item
    x2 = np.empty((1, 30, 3)) # sales, year trig
    y = np.empty((1, 1, 1))

    val1 = [[]] # store, item
    val2 = np.empty((1, 30, 3)) # sales, year trig
    valy = np.empty((1, 1, 1))

    for store in tqdm(stores):
        temps = data[(data["store"]==store)]
    
        items = temps["item"].unique()

        for item in items:
            temp = temps[(temps["item"]==item)].copy()
            temp.drop(labels=["store", "item"], axis=1, inplace=True)

            length = temp.__len__()
            stack = np.empty((1, 31, 3))

            for index in range(length - 30):
                array = np.array(temp[index:index+31])
                stack = np.append(stack, array.reshape(1, -1, 3), 0)

            stack = stack[1:, :, :]

            stack = tf.convert_to_tensor(stack)
            inputs, labels = window.split_window(stack)

            shp = inputs.shape[0]
            train_size = shp*8//10
            val_size = shp - train_size

            x1.extend([[store, item]]*train_size)
            x2 = tf.concat([x2, inputs[:train_size]], 0)
            y = tf.concat([y, labels[:train_size]], 0)

            val1.extend([[store, item]]*val_size)
            val2 = tf.concat([val2, inputs[train_size:]], 0)
            valy = tf.concat([valy, labels[train_size:]], 0)
                
    x1 = np.array(x1[1:])
    x2 = x2.numpy()[1:, :, :]
    y = y.numpy()[1:, :, :].reshape(-1, 1)

    val1 = np.array(val1[1:])
    val2 = val2.numpy()[1:, :, :]
    valy = valy.numpy()[1:, :, :].reshape(-1, 1)

    train_data = tf.data.Dataset.from_tensor_slices(((x1, x2), y)).batch(BATCH_SIZE)
    val_data = tf.data.Dataset.from_tensor_slices(((val1, val2), valy)).batch(valy.shape[0])

    return train_data, val_data

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import keras

TIME_DELAY = 30

def create_model() -> keras.Model:
    input_1 = keras.Input((2,), name="input_1")
    input_2 = keras.Input((TIME_DELAY, 3), name="input_2")
    dense = keras.layers.Dense(2)(input_1)
    flatten1 = keras.layers.Flatten()(dense)
    lstm = keras.layers.LSTM(TIME_DELAY, return_sequences=True)(input_2)
    flatten2 = keras.layers.Flatten()(lstm)

    concat = keras.layers.concatenate([flatten1, flatten2])
    dense = keras.layers.Dense(256, activation="relu")(concat)
    dropout = keras.layers.Dropout(0.2)(dense)
    dense = keras.layers.Dense(64, activation="relu")(dropout)
    dropout = keras.layers.Dropout(0.2)(dense)
    output = keras.layers.Dense(1)(dropout)
    return keras.Model(inputs=[input_1, input_2], outputs=output)

In [3]:
MAX_EPOCH = 50

df = pd.read_csv("train.csv")
model = create_model()
model.summary()

train_data, val_data = make_data(df)

early_stopper = tf.keras.callbacks.EarlyStopping("val_loss", 0.1, 4, 1, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_delta=0.3, min_lr=0)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("tmp/models", "val_loss", 1, True)
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer, "mse", ["mse", "mae"])

model.fit(train_data, batch_size=BATCH_SIZE, epochs=MAX_EPOCH, 
          callbacks=[early_stopper, reduce_lr, model_checkpoint], validation_data=val_data)

model.evaluate(val_data)

model.save('models/my_model.h5')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 30, 3)]      0           []                               
                                                                                                  
 dense (Dense)                  (None, 2)            6           ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    (None, 30, 30)       4080        ['input_2[0][0]']                
                                                                                              

100%|██████████| 10/10 [04:03<00:00, 24.36s/it]


Epoch 1/50
44873/44875 [============================>.] - ETA: 0s - loss: 133.5685 - mse: 133.5685 - mae: 8.6169
Epoch 1: val_loss improved from inf to 133.72977, saving model to tmp/models


INFO:tensorflow:Assets written to: tmp/models/assets


INFO:tensorflow:Assets written to: tmp/models/assets


...

INFO:tensorflow:Assets written to: tmp/models/assets


44875/44875 [==============================] - 227s 5ms/step - loss: 68.0638 - mse: 68.0638 - mae: 6.2968 - val_loss: 74.1558 - val_mse: 74.1558 - val_mae: 6.6032 - lr: 6.2500e-05
Epoch 20/50
44868/44875 [============================>.] - ETA: 0s - loss: 68.0228 - mse: 68.0228 - mae: 6.2965
Epoch 20: val_loss did not improve from 74.15583
44875/44875 [==============================] - 222s 5ms/step - loss: 68.0320 - mse: 68.0320 - mae: 6.2969 - val_loss: 75.1944 - val_mse: 75.1944 - val_mae: 6.6320 - lr: 6.2500e-05
Epoch 21/50
44871/44875 [============================>.] - ETA: 0s - loss: 67.7855 - mse: 67.7855 - mae: 6.2873
Epoch 21: val_loss did not improve from 74.15583
44875/44875 [==============================] - 224s 5ms/step - loss: 67.7871 - mse: 67.7871 - mae: 6.2874 - val_loss: 75.3061 - val_mse: 75.3061 - val_mae: 6.6380 - lr: 6.2500e-05
Epoch 22/50
44865/44875 [============================>.] - ETA: 0s - loss: 67.8376 - mse: 67.8376 - mae: 6.2882
Epoch 22: val_loss improve

INFO:tensorflow:Assets written to: tmp/models/assets


INFO:tensorflow:Assets written to: tmp/models/assets


44875/44875 [==============================] - 229s 5ms/step - loss: 67.8481 - mse: 67.8481 - mae: 6.2887 - val_loss: 73.0842 - val_mse: 73.0842 - val_mae: 6.5655 - lr: 3.1250e-05
Epoch 23/50
44868/44875 [============================>.] - ETA: 0s - loss: 67.7728 - mse: 67.7728 - mae: 6.2854
Epoch 23: val_loss did not improve from 73.08415
44875/44875 [==============================] - 223s 5ms/step - loss: 67.7765 - mse: 67.7765 - mae: 6.2856 - val_loss: 74.3549 - val_mse: 74.3549 - val_mae: 6.6077 - lr: 3.1250e-05
Epoch 24/50
44870/44875 [============================>.] - ETA: 0s - loss: 67.7823 - mse: 67.7823 - mae: 6.2857
Epoch 24: val_loss did not improve from 73.08415
44875/44875 [==============================] - 225s 5ms/step - loss: 67.7858 - mse: 67.7858 - mae: 6.2859 - val_loss: 73.3960 - val_mse: 73.3960 - val_mae: 6.5770 - lr: 3.1250e-05
Epoch 25/50
44875/44875 [==============================] - ETA: 0s - loss: 67.7476 - mse: 67.7476 - mae: 6.2826
Epoch 25: val_loss did not